In [ ]:
import math, random
import gc, os
import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys

sys.path.append("../input/iterative-stratification/iterative-stratification-master")
sys.path.append("../input/modules/datasets")
sys.path.append("../input/modules/facebookresearch")
sys.path.append("../input/modules/Qwicen")
sys.path.append("../input/modules/trainer")
sys.path.append("../input/modules/utils")
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from Tab_dataset import MoaDataset
from node import NODE
from preprocess import *
from qhoptim import QHAdam
from tab_trainer import mean_log_loss
from tab_trainer import TabTrainer

import warnings
warnings.filterwarnings('ignore')
%load_ext autoreload
%autoreload 2

In [ ]:
train_features = pd.read_csv("../input/lish-moa/train_features.csv")
train_targets = pd.read_csv('../input/lish-moa/train_targets_scored.csv')
train_nontargets = pd.read_csv('../input/lish-moa/train_targets_nonscored.csv')

test_features = pd.read_csv('../input/lish-moa/test_features.csv')
ss = pd.read_csv('../input/lish-moa/sample_submission.csv')

In [ ]:
GENES = [col for col in train_features.columns if col.startswith("g-")]
CELLS = [col for col in train_features.columns if col.startswith("c-")]
config = {"QuantileTransformer": {"random_state": 1, "n_quantiles": 1000, "output_distribution": "normal"}}
# train_features, test_features = apply_rank_gauss(
#         train_features, test_features, columns=GENES + CELLS, config=config
#     )
train_features, test_features = apply_zscore(train_features, test_features, columns=GENES + CELLS)
SEED = 1
train_features, test_features = apply_pca(train_features, test_features, GENES, threshold=0.9, kind="g", SEED=1)
train_features, test_features = apply_pca(train_features, test_features, CELLS, threshold=0.9, kind="c", SEED=1)

In [ ]:
pca = PCA(random_state=SEED)
columns = CELLS
pca.fit(train_features[columns])
train2 = pca.transform(train_features[columns])
test2 = pca.transform(test_features[columns])

In [ ]:
pca_cols = [col for col in train_features.columns if col.startswith("pca")]
train_features, test_features = apply_zscore(
    train_features, test_features, pca_cols
)

In [ ]:
plt.plot(pca.explained_variance_ratio_.cumsum())
sum(pca.explained_variance_ratio_.cumsum()<0.9)

In [ ]:

train_features, test_features = apply_zscore(train_features, test_features, list(test_features.columns)[4:])

In [ ]:
plt.scatter(test_features.loc[:, "pca_g-0"], test_features.loc[:, "pca_g-1"])

In [ ]:
from sklearn.mixture import BayesianGaussianMixture
dpgmm = BayesianGaussianMixture(n_components=30, covariance_type='diag', max_iter=100, random_state=1, n_init=1, weight_concentration_prior_type="dirichlet_process")
dpgmm.fit(train_features.iloc[:, 4:])

In [ ]:
train_proba = dpgmm.predict_proba(train_features.iloc[:, 4:])
test_proba = dpgmm.predict_proba(test_features.iloc[:, 4:])

In [ ]:
plt.plot(range(30), dpgmm.weights_)

In [ ]:
sum(dpgmm.weights_ > 0.02)

In [ ]:
import joblib
from preprocess import create_dpgmm_proba
train_features, test_features = create_dpgmm_proba(
            train_features,
            test_features,
            GENES,
            path="exp/node/v008/dpgmm_zscore_g.job",
            config={},
            kind="g",
        )



In [ ]:
train_clusters_g = train_features.loc[:, [f"dpgmm_g-{i}" for i in range(40)]]
test_clusters_g = test_features.loc[:, [f"dpgmm_g-{i}" for i in range(40)]]

plt.figure(figsize=(16, 8))
plt.subplot(2, 1, 1)
plt.imshow(train_clusters_g.values, aspect="auto")
plt.subplot(2, 1, 2)
plt.imshow(test_clusters_g.values, aspect="auto")
# plt.colorbar()

In [ ]:
train_features, test_features = create_dpgmm_proba(
            train_features,
            test_features,
            CELLS,
            path="exp/node/v009/dpgmm_RankGauss_c.job",
            config={},
            kind="c",
        )

In [ ]:
train_clusters_c = train_features.loc[:, [f"dpgmm_c-{i}" for i in range(20)]]
test_clusters_c = test_features.loc[:, [f"dpgmm_c-{i}" for i in range(20)]]

plt.figure(figsize=(16, 8))
plt.subplot(2, 1, 1)
plt.imshow(train_clusters_c.values, aspect="auto")
plt.subplot(2, 1, 2)
plt.imshow(test_clusters_c.values, aspect="auto")
# plt.colorbar()

In [ ]:
with open("exp/node/v009/dpgmm_RankGauss_c.job", "rb") as f:
    dpgmm_c = joblib.load(f)
 